In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from itertools import product
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [2]:
#exten the all data
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
trainsets = pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/train.csv")
testsets = pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/test.csv")

In [79]:
#deal with the all dataset 
x_train=trainsets.iloc[:, :80].set_index('Id')
y_train=trainsets.iloc[:,-1]
x_test=testsets.iloc[:, :80].set_index('Id')

In [ ]:
dataset.contact

In [94]:
#import the variable sheet
vars_info=pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/variable_info.csv", index_col=0)
vars_info=vars_info.T
vars_info.head()
#because the SalePrice is Y, I drop the SalePrice.
vars_info=vars_info.drop(columns=[80], axis=1)
vars_info= vars_info.dropna(axis = 1)
vars_info=vars_info.T
vars_info

,Id,correlate,continuous,Chinese_notes
1,MSSubClass,0,1,房子的類型（適合怎樣的家庭組成）
2,MSZoning,0,0,土地利用情形
3,LotFrontage,0,0,連接幾吋街道
4,LotArea,0,1,有幾平方英尺
5,Street,0,0,鄰近街道的鋪面
6,Alley,0,0,房屋外觀（規則或不規則形)
7,LotShape,0,0,房屋平坦程度
8,LandContour,0,0,房屋鄰近山坡
9,Utilities,0,0,公共資源（水電瓦斯污水）
11,LandSlope,0,0,房屋坡度


In [95]:
#groups the data for corrleate or continuous
groups = {}
i = [0,1]
j = [0,1]
for x, y in product(i, j):
    tmp = vars_info[(vars_info.correlate == x).to_numpy() & (vars_info.continuous == y).to_numpy()]['Id'].to_numpy()
    groups[f'group_{x}_{y}'] = tmp


In [98]:
groups['group_0_0'].tolist()

['MSZoning',
 'LotFrontage',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'E0terior1st',
 'E0terior2nd',
 'MasVnrType',
 'MasVnrArea',
 'E0terQual',
 'E0terCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtE0posure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [118]:
aa=x_train.columns.isin(groups['group_0_0'])
type(aa.tolist()[0])


bool

None


In [97]:
#clean data
correlation=x_train.loc[:, groups['group_0_0'].tolist()]

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'